In [32]:
import pandas as pd
import re
import requests
import os.path
import collections
import collections
import spacy
from os import listdir
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
from __future__ import unicode_literals, print_function
from spacy.lang.en import English
import random
import json
import unicodedata
import requests

In [33]:
material_id={}

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)#codec=codec,
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text.decode("utf-8")

In [62]:
def temp_extractor(str):
    dictionary={}
    
    #
    #[\d+ ,]*to*[ ]*\d+ °C 
    temp_reg = re.findall('(\d+[^\d]{1,4}\d*)\s*°C', str)
    if len(temp_reg) > 0:

        temp_reg = [temp + ' °C' for temp in temp_reg]
        #print('temperature 1',temp_reg)
        
        dictionary[('temperature').encode('utf-8')]=temp_reg
        temp_reg=''
    
    
    
    
    temp_reg = re.findall('([\d+ ,to]*[ ]*\d+)\s*°C', str)
    if len(temp_reg) > 0:
        temperature_list=[]
        #print('\n ##############Temperature###############')
        #temp_reg = [unicodedata.normalize('NFKD', temp).decode('ascii','ignore') + " 'C" for temp in temp_reg]
        #temp_reg = [temp + " 'C" for temp in temp_reg]
        #temp_reg=temp_reg.decode('utf-8')
        for tem in temp_reg:
            tm=tem
            if tm.find('to')!=-1:
                if len(tm)>7:
                    
                    print('\n toto is calling :',tm,'after : ',tm.replace("to","-"))
                    tm=tm.replace("to","-")
                    tm=tm+' °C'
                    temperature_list.append(tm.replace("to","-"))
                else:
                    temperature_list.append(tm.replace("to",""))
                #print('\n its find to',tm.replace("to",""))
            elif tm.find('t')!=-1:
                
                tm=tm.replace("t","")
                tm=tm+' °C'
                temperature_list.append(tm.replace("t",""))
                #print('\n its find T',tm.replace("t",""))
                
            elif tm.find('o')!=-1:
                tm=tm.replace("o","")
                tm=tm+' °C'               
                
                temperature_list.append(tm.replace("o",""))
                #print('\n its find O',tm.replace("o",""))
            else:
                temperature_list.append(tm)
                #print('not contain ',tm)
        #print('temperature 1',temp_reg)

        dictionary['temperature']=temperature_list
        temp_reg=''

        
        
        
    
#     temp_reg = re.findall('([[^to]\s*\d+)\s*°C', str)
#     if len(temp_reg) > 0:
#         print('\n @@@ not to Temperature@@@@',temp_reg)
#         #temp_reg = [unicodedata.normalize('NFKD', temp).decode('ascii','ignore') + " 'C" for temp in temp_reg]
#         temp_reg = [temp + " 'C" for temp in temp_reg]

#         #print('temperature 1',temp_reg)

#         dictionary['temperature']=temp_reg
#         temp_reg=''

                
        
        
        
        
    temp_reg = re.findall('(\d+) K[.]*', str)
    if len(temp_reg) > 0:
        #print('temperature 2',temp_reg)
        
        temp_reg = [temp + " K" for temp in temp_reg]
        dictionary['temperature']=temp_reg
        temp_reg=''


        
    temp_reg = re.findall('(\d+) �C', str)

    if len(temp_reg) > 0:
        #print('temperature',temp_reg)

        temp_reg = [temp + " °C" for temp in temp_reg]
        dictionary['temperature']=temp_reg
        temp_reg=''


        
        
    temp_reg = re.findall('(\d+)s-1|(\d+) s-1|\d+ s -1|\d+ s 1', str)

    if len(temp_reg) > 0:
        #print('strain_rate is calling',temp_reg)

        temp_reg = [temp + " s-1" for temp in temp_reg]
        dictionary['strain_rate']=temp_reg
        temp_reg=''


        


        
    temp_reg = re.findall('[\d.]*\d+ s|\d+ h[ .]+|\d+ m$|\d+ min', str)

    if len(temp_reg) > 0:
        #print('time',temp_reg)
        
        temp_reg = [temp for temp in temp_reg]
        dictionary[('time').encode('utf-8')]=temp_reg
        temp_reg=''


        
        
#     temp_reg = re.findall('[/d+ -,]*\d+ mm |[/d+ -,]*\d+ nm', str) #to will use

#     if len(temp_reg) > 0:
#         #print('height',temp_reg)
        
#         temp_reg = [temp + "mm" for temp in temp_reg]
#         dictionary[('height').encode('utf-8')]=temp_reg
#         temp_reg=''

    temp_reg = re.findall('([/d+ -,]*\d+) K/s', str)

    if len(temp_reg) > 0:
        #print('heating rate',temp_reg)

        temp_reg = [temp + " K/s" for temp in temp_reg]
        dictionary[('heating_rate').encode('utf-8')]=temp_reg
        temp_reg=''

        

    temp_reg = re.findall('([\d+ -,.]*\d+) kJ mol', str)

    if len(temp_reg) > 0:
        #print('energy ',temp_reg)

        temp_reg = [temp + " kJ mol" for temp in temp_reg]
        dictionary[('energy').encode('utf-8')]=temp_reg
        temp_reg=''
        
        
        
#kJ mol−1        

#|((\d+) K)+((\d+) s$)  
        
#2 K/s height


    return dictionary
def microstructure_extractor(str):
    dictionary={}
    temp_reg = re.findall(('([\d+ -,to]*\d+) μm'), str)

    if len(temp_reg) > 0:
        print('\n **********grain_size*****************',temp_reg)
        #print('\n sentences ',str)

        temp_reg = [temp for temp in temp_reg]
        dictionary[('grain_size').encode('utf-8')]=temp_reg
        temp_reg=''    
    return dictionary


def properties_extractor(str):
    dictionary={}
    
    temp_reg = re.findall('[\d+ -,.]*\d+ mV/s', str)

    if len(temp_reg) > 0:
        
        print('\n polarisation ',temp_reg)
#         print('\n sentences ',str)
        
        temp_reg = [temp  for temp in temp_reg]
        dictionary['polarisation']=temp_reg
        temp_reg=''

#HV

    temp_reg = re.findall('[\d+ -,to]*\d+ [to ]*HV[,]*', str)

    if len(temp_reg) > 0:
        print('\n hardness ',temp_reg)
#         print('\n sentences ',str)

        temp_reg = [temp for temp in temp_reg]
        dictionary[('hardness').encode('utf-8')]=temp_reg
        temp_reg=''
    





    temp_reg = re.findall('[\d+ -,to]*\d+ [to ]*HV1[,]*', str)

    if len(temp_reg) > 0:
        
        #print('\n hardness ',temp_reg)
#         print('\n sentences ',str)

        temp_reg = [temp for temp in temp_reg]
        dictionary[('hardness').encode('utf-8')]=temp_reg
        temp_reg=''
    

    
    
    return dictionary




In [63]:


def get_process(proc_doc,sentence,mat_id):
    for ent in proc_doc.ents:

        res=temp_extractor(sentence)
        if len(res)!=0:
            res['mat_id']=mat_id
            res[str('process')]=str(ent).encode('utf-8')
            return res.copy()
            
    return 0



def get_dict_process(process_list,sentence,mat_id):
    
    for process in process_list:

    #print('process name :',process)

        if re.search(process,sentence):
            process_dict=temp_extractor(sentence)
            if len(process_dict)!=0:
                process_dict['mat_id']=mat_id
                process_dict[str('process')]=process
                return process_dict.copy()
#                     process_dict.clear()
    return 0

def get_composition(comp_doc):
    for composition in comp_doc.ents:

        if len(str(composition)) >6:
            c1=unicodedata.normalize('NFKD', composition.string).encode('ascii','ignore')
            return c1
    return 0    
def get_caption(sentence):
    if re.search('Fig[ .\n\d+]*',sentence):
        return sentence
    return 0




            
def get_devices(doc_devices):
    for device in doc_devices.ents:
        if len(str(device)) >3:
            return str(device)
    return 0



def get_dictionary(processes_list,properties_list,devices_list,compositions_list,microstructure_list):
    temp_dict={}
    temp_dict['processes']=processes_list 
    temp_dict['compositions']=compositions_list 
    temp_dict['devices']=devices_list 
    temp_dict['properties']=properties_list 
    temp_dict['microstructures']=microstructure_list 

    return temp_dict.copy()





def get_properties(prop_doc,sentence,mat_id):
    for prop in prop_doc.ents:
        prop_dict=properties_extractor(sentence)
        if len(prop_dict)!=0:
            prop_dict['mat_id']=mat_id
            prop_dict[str('properties')]=str(prop).encode('utf-8')
            return prop_dict.copy()
        
        
    return 0    




def get_dict_properties(properties_list,sentence,mat_id):
    
    for prop in properties_list:
        if re.search(prop,sentence):
            prop_dict=properties_extractor(sentence)
            if len(prop_dict)!=0:
                prop_dict['mat_id']=mat_id
                prop_dict[str('properties')]=prop
                return prop_dict.copy()
    return 0




def get_dict_microstructure(microstructure_list,sentence,mat_id):
    for micro in microstructure_list:
        if re.search(micro,sentence):
            micro_dict=microstructure_extractor(sentence)
            if len(micro_dict)!=0:
                micro_dict['mat_id']=mat_id
                micro_dict[str('microstructure')]=micro
                return micro_dict.copy()
#                     process_dict.clear()
    return 0



#microstructure_extractor

In [71]:
def paper_extractor(path,process_loc,prop_loc,device_loc,comp_loc):
    data_dict={}
    process_list=['heat treated','quenched','heated','homogenized','Sensitisation','deformation','recrystallisation','sensitisation','polished','brushing','sensitised','stirred','melt','graphite','solutionised','thermomechanical','polished','normalisation','deformed','diffusion','unrecrystallised','corrosion','homogenized']
    manual_property_list=['polarisation','hardness']
    manual_microstructure_list=['grain size']
    url_backend="http://192.168.23.74:7711/postSchemaDict" 
    process_finder=False
#    material_id={}
    
#     material_id['1-s2.0-S026130691100553X-main']='2219'
#     material_id['1-s2.0-S0261306911002809-main']='AA7050'
#     material_id['1-s2.0-S0924013615300029-main']='AA7050'
#     ## adding
#     material_id['1-s2.0-S0921509313001275-main']='AA7150'
#     material_id['1-s2.0-S0921509313008769-main']='Al-Cu-Li'
#     material_id['1-s2.0-S0921509315305190-main']='Al-5083'
#     material_id['1-s2.0-S0921509315000738-main']='Al-7Mg'
#     material_id['1-s2.0-S0925838815304242-main']='2060'
#     material_id['1-s2.0-S0925838817304140-main']='Al-5083'
#     material_id['1-s2.0-S1003632617600335-main']='Al-9.0Mg-0.5Mn-0.1Ti'

    #manual_property_list=['polarisation','hardness','grain size']
    #m_id=['ASN','AA5083','Al-xSr-yNd-Mg','Al-xMn-Mg','Al-xSr-yNd-Mg','Mg-Zn-Ca','Mg-Ce-La-Nd','Mg-Zn-Ca-Zr-Gd-Sr','AA7075-T651','AM1','AM2','AM3','AM4','ASN1','ASN2','ASN3','ASN4','ASN5','ASN6','ASTM G67-04']
    nlp = spacy.load("en")
    prdnlp=spacy.load(process_loc)
    properties=spacy.load(prop_loc)
    devices=spacy.load(device_loc)
    compositions=spacy.load(comp_loc)

    sentencizer = nlp.create_pipe("sentencizer")
    #nlp.add_pipe(sentencizer, last=True)
    nlp.add_pipe(sentencizer, before="parser")    
    pdf_files=listdir(path)
    microstructure_list=[]
    microstructure_dict_list=[]
    properties_list=[]
    property_dict_list=[]
    processes_list=[]
    process_dict_list=[]
    devices_list=[]
    #compositions_list=[]
    process_find_list=[]
    caption_list=[]

    loop_breaker_process=False
    loop_breaker_properties=False
    
    
    
    
    
    
    for pdf_file_index in range(0,len(pdf_files)):
        pdf_file_path=path+pdf_files[pdf_file_index]
        print('\n Address :',pdf_file_path)
        compositions_list=[]
        text_file=convert_pdf_to_txt(pdf_file_path)
        compositions_list=re.findall('AA[\d\w]*',text_file)
        compositions_list=list(set(compositions_list))
        if len(compositions_list)<1:
            compositions_list=[]
        
        doc = nlp(text_file) 
        
        ############Start_composition###############
        for sent in doc.sents:
            sentence=sent.text
            doc_compositions = compositions(sentence)
            
            composition_result=get_composition(doc_compositions)
            
            
            if composition_result!=0:
                
                compositions_list.append(composition_result)

        if len(compositions_list)==0:
            
            compositions_list=['UC']
            
        compositions_list=set(compositions_list)
        
        ############End_composition##################
        for sent in doc.sents:
            
            sentence=sent.text
            #print('sentence_type :',type(sentence))
            
            dic_length=True
            #print(sentence)
            doc1 = prdnlp(sentence)
            doc_prop = properties(sentence)
            doc_devices = devices(sentence)
            doc_compositions = compositions(sentence)


    ########################Caption_Start####################################
            #print(sentence)
            capiton_result=get_caption(sentence)
            if capiton_result!=0:
                caption_list.append(capiton_result)
                

    ########################Caption_End######################################      

    ########################Processes_start############################################
            #print('\n processes is calling data :',doc1)
            #print('\n process is just calling')
            if len(compositions_list)>0:
                for comp in compositions_list:
                    
                    process_result=get_process(doc1,sentence,comp)
                    if process_result!=0:

                        processes_list.append(process_result)

    ########################process End#############################################


    #########################process regex Start ###################################
#             process_dict_result=get_dict_process(process_list,sentence,material_id[pdf_files[pdf_file_index][:-4]])
#             if process_dict_result!=0:
#                 process_dict_list.append(process_dict_result)

    ########################process regex End########################################
 
    ####Property Regex
#             property_dict_result=get_dict_properties(manual_property_list,sentence,material_id[pdf_files[pdf_file_index][:-4]])
#             if property_dict_result!=0:
#                 #print('property_dict_result : ',property_dict_result)
#                 property_dict_list.append(property_dict_result)

#             #microstructure_dict_list
#             microsturcture_dict_result=get_dict_microstructure(manual_microstructure_list,sentence,material_id[pdf_files[pdf_file_index][:-4]])
#             if microsturcture_dict_result!=0:
                
#                     microstructure_dict_list.append(microsturcture_dict_result)
                
                
    
    #########################property  Start ######################################## 
    
    
            if len(compositions_list)>0:
                for comp in compositions_list:


                    property_result=get_properties(doc_prop,sentence,comp)
                    if property_result!=0:
                        properties_list.append(property_result)    
    ########################property End##############################################
    
    
    ################################Devices Start ####################################
            device_result=get_devices(doc_devices)
            if device_result!=0:
                devices_list.append(device_result)
    ################################Devices End #######################################

    ################################Composition Start #######################################
    
#             composition_result=get_composition(doc_compositions)
#             if composition_result!=0:
                
#                 compositions_list.append(composition_result)
                
    ################################Composition End #######################################
                
                
        #print('composition_list',compositions_list)
        #print('process_dict_list',process_dict_list)
        ###For each paper
        #data_dict=get_dictionary(processes_list,property_dict_list,devices_list,compositions_list,manual_microstructure_list)
        data_dict=get_dictionary(processes_list,processes_list,devices_list,compositions_list,manual_microstructure_list)
        print('process :',data_dict['processes'])
        ####Sending Api for each paper
        #requests.post(url_backend, json={'schema_dict': data_dict,'file_name':pdf_files[pdf_file_index][:-4]})
        #print('\n devices',devices_list)
        processes_list=[]
        processes_list=[]
        compositions_list=[]
        devices_list=[]
        property_dict_list=[]  
        processes_list
        manual_microstructure_list=[]


    #For All papers
    #data_dict=get_dictionary(processes_list,properties_list,devices_list,compositions_list)
    return data_dict

In [72]:
process_loc="Processes/Processes_GPU_model_0.2"
prop_loc="Properties/Properties_GPU_model_0.2"
device_loc="Methods/Methods_small_GPU_model_0.1"
comp_loc="Compositions/Compositions_GPU_model_0.1"
#path="/home/cloud4/Mehroz/Mat_Sci/one_paper_test/"
path="/home/cloud4/Mehroz/Mat_Sci/process_finding/"


dictionary=paper_extractor(path,process_loc,prop_loc,device_loc,comp_loc)




 Address : /home/cloud4/Mehroz/Mat_Sci/process_finding/1-s2.0-S0261306911002809-main.pdf
process : [{'temperature': ['573 K'], 'mat_id': 'AA7050', 'process': b'deformation'}, {'temperature': ['573 K'], 'mat_id': 'AA7010', 'process': b'deformation'}, {'temperature': ['673 K'], 'mat_id': 'AA7050', 'process': b'deformation'}, {'temperature': ['673 K'], 'mat_id': 'AA7010', 'process': b'deformation'}, {'temperature': ['753 K', '756 K'], b'time': ['16 h '], 'mat_id': 'AA7050', 'process': b'solution treated'}, {'temperature': ['753 K', '756 K'], b'time': ['16 h '], 'mat_id': 'AA7010', 'process': b'solution treated'}, {'temperature': ['2 K'], b'time': ['10 s'], 'mat_id': 'AA7050', b'heating_rate': ['2 K/s'], 'process': b'heating rate'}, {'temperature': ['2 K'], b'time': ['10 s'], 'mat_id': 'AA7010', b'heating_rate': ['2 K/s'], 'process': b'heating rate'}, {'temperature': ['2 K', '573 K'], b'time': ['120 s', '10 s'], 'mat_id': 'AA7050', b'heating_rate': [' 2 K/s'], 'process': b'heated'}, {'tem


 hardness  ['5 HV']
process : [{b'temperature': ['1087  °C'], 'temperature': [' 1087'], 'mat_id': 'UC', 'process': b'melting'}]

 Address : /home/cloud4/Mehroz/Mat_Sci/process_finding/1-s2.0-S0921509315000738-main.pdf
process : [{b'time': ['103 s'], 'mat_id': 'UC', 'process': b'deformation'}, {b'time': ['3 s'], 'mat_id': 'UC', 'process': b'deformed'}, {b'time': ['6 s'], 'mat_id': 'UC', 'process': b'deformed'}]

 Address : /home/cloud4/Mehroz/Mat_Sci/process_finding/1-s2.0-S1003632617600335-main.pdf
process : [{b'temperature': ['300−450 °C'], 'temperature': ['450'], 'mat_id': b'AlMgSnHg ', 'process': b'deformation'}, {b'temperature': ['300−450 °C'], 'temperature': ['450'], 'mat_id': b'Al9.0Mg0.5Mn ', 'process': b'deformation'}, {b'temperature': ['300−450 °C'], 'temperature': ['450'], 'mat_id': b'AlMgLiZr ', 'process': b'deformation'}, {b'temperature': ['300−450 °C'], 'temperature': ['450'], 'mat_id': 'AA7085', 'process': b'deformation'}, {b'temperature': ['300−450 °C'], 'temperature': 

process : [{b'time': ['10 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'deformation'}, {b'time': ['24 h '], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'homogenized'}, {b'time': ['10 s', '5 min'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'heated'}, {b'time': ['10 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'stress'}, {b'time': ['01 s', '1 s', '1 s', '10 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'stress'}, {b'time': ['7 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'deformed'}, {b'time': ['10 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'deformed'}, {b'time': ['0.01 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'deformed'}, {b'time': ['0.01 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0.7Mge0.4Age0.12Zr ', 'process': b'deformed'}, {b'time': ['11 s'], 'mat_id': b'Ale3.6Cue0.8Lie0.4Zne0

In [30]:
dictionary

[{b'time': ['10 s'],
  'mat_id': 'AA7150',
  'process': b'stress',
  'temperature': ['723K']},
 {'mat_id': 'AA7150', 'process': b'homogenized', 'temperature': ['738K']},
 {b'heating_rate': [' 10s-1'],
  b'time': ['10 s', '3 min'],
  'mat_id': 'AA7150',
  'process': b'compression tests',
  'temperature': ['723K', '10K']},
 {b'time': ['10 s'],
  'mat_id': 'AA7150',
  'process': b'stress',
  'temperature': ['723K']},
 {b'time': ['10 s', '0.001 s'],
  'mat_id': 'AA7150',
  'process': b'stress',
  'temperature': ['573K', '723K']},
 {b'time': ['10 s', '10 s'],
  'mat_id': 'AA7150',
  'process': b'deformed',
  'temperature': ['573K', '673K']},
 {b'time': ['0.01 s'],
  'mat_id': 'AA7150',
  'process': b'deformation',
  'temperature': ['673K']},
 {b'time': ['10 s', '0.001 s'], 'mat_id': 'AA7150', 'process': b'stress'},
 {b'time': ['1 s', '0.1 s', '0.01 s'],
  'mat_id': 'AA7150',
  'process': b'deformation',
  'temperature': ['673K']},
 {b'time': ['10 s'],
  'mat_id': 'AA7150',
  'process': b'de